In [9]:
from pynq import Overlay
from pynq.lib import DMA
from pynq import MMIO
from pynq import Xlnk
import numpy as np


ol = Overlay("/home/xilinx/rosetta/design_1.bit")
ol.download()
#print(ol.ip_dict)
#dma_read_addr = ol.ip_dict["axi_dma_0"]["phys_addr"]

dma1 = ol.axi_dma_0
dma2 = ol.axi_dma_2

#dma3 = ol.axi_dma_2

#dma_s = MMIO(dma_read_addr, 128)



In [10]:
from PIL import Image#, ImageFilter
from time import time
#Read image

start = time()
im = Image.open( '/home/xilinx/rosetta/image.jpg' ).resize((320,240))
im.save("image_resized.png", "PNG")
out_data = []
in_data = list(im.getdata())

print(im.size)

R, G, B = 0, 1, 2

for pixel in in_data:
    out_data.append(pixel[R])
    out_data.append(pixel[G])
    out_data.append(pixel[B])

numberOfData = im.size[0]*im.size[1]*3
numberOfDataBlur = (im.size[0]-2)*(im.size[1]-2)
xlnk = Xlnk()

gray1_in_buffer = xlnk.cma_array(shape=(numberOfData,))
gray2_in_buffer = xlnk.cma_array(shape=(numberOfData,))

blur_out_buffer = xlnk.cma_array(shape=(numberOfDataBlur,))
gray2_out_buffer = xlnk.cma_array(shape=(numberOfData//3,))


for i in range(0, numberOfData):
    
    gray2_in_buffer[i] = out_data[i]
    #out1_buffer[i] = 0

for i in range(0, numberOfData):#numberOfData//3):
    #gray1_in_buffer[i] = int((in_data[i][R]*(0.3)+in_data[i][G]*(0.59)+in_data[i][B]*(0.11)))<<16
    gray1_in_buffer[i] = out_data[i]

end = time()
print("Before dma.send took: ",end-start)


dma1.sendchannel.transfer(gray1_in_buffer)
dma1.recvchannel.transfer(blur_out_buffer)

dma2.sendchannel.transfer(gray2_in_buffer)
dma2.recvchannel.transfer(gray2_out_buffer)


#dma1.sendchannel.wait()
  
start = time()

blurim = Image.new("RGB", (im.size[0]-2, im.size[1]-2))
grayim = Image.new("RGB", im.size)
gray_data = []
blur_data = []
for i in range(0, numberOfDataBlur):
    channel_blur = blur_out_buffer[i]#>>16 #+100 
    #pixel = (out1_buffer[i]+70, out1_buffer[i]+70, out1_buffer[i]+70)
    #data.append((out1_buffer[i]+50, out1_buffer[i]+50, out1_buffer[i]+50))
    blur_data.append((channel_blur, channel_blur, channel_blur))
    #if (i<320):
    #    print((channel_blur, channel_blur, channel_blur))
#print("==============================================")
for i in range(0, numberOfData//3):
    channel_gray = gray2_out_buffer[i] #+100 
    #pixel = (out1_buffer[i]+70, out1_buffer[i]+70, out1_buffer[i]+70)
    #data.append((out1_buffer[i]+50, out1_buffer[i]+50, out1_buffer[i]+50))
    
    gray_data.append((channel_gray, channel_gray, channel_gray))
    #if (i<320):
    #    print((channel_gray,channel_gray,channel_gray))
       

    
    
blurim.putdata(blur_data)
blurim.save('image_blurred.png', 'PNG')

grayim.putdata(gray_data)
grayim.save('image_grayscale.png', 'PNG')

end = time()
print("Rebuilding image took: ", end-start)

gray1_in_buffer.close()
gray2_in_buffer.close()
blur_out_buffer.close()
gray2_out_buffer.close()

(320, 240)
Before dma.send took:  3.3251144886016846
Rebuilding image took:  1.7529816627502441


In [16]:
xlnk = Xlnk()
in_buffer = xlnk.cma_array(shape=(numberOfData,))

out1_buffer = xlnk.cma_array(shape=(numberOfData,), dtype=np.int32)#, dtype=np.int32)
#out2_buffer = xlnk.cma_array(shape=(1000, ),dtype=np.int32)#, dtype=np.int32)
#out3_buffer = xlnk.cma_array(shape=(1000,), dtype=np.int32)

In [17]:
for i in range(0, numberOfData):
    in_buffer[i] = out_data[i]
    out1_buffer[i] = 0;
    #out2_buffer[i] = 0;
    #out3_buffer[i] = 0;

#for i in range(0, 20):
    #print(out1_buffer[i]," <- out1, out2->", out2_buffer[i])
    


In [18]:

#dma1.sendchannel.transfer(in_buffer)
#dma1.recvchannel.transfer(out1_buffer)


#dma.sendchannel.transfer(in_buffer)
#dma2.recvchannel.transfer(out2_buffer)
#dma3.recvchannel.transfer(out3_buffer)
#dma.sendchannel.wait()
#dma.recvchannel.wait()



In [ ]:
for i in range(2, len(out_data)):
    #R, G, B, R, G ,B .... layout of the data in the 1d array
    source[R].putpixel(())
    
    

In [2]:
#for i in range(0, 20):
    #print(in_buffer[i]," <- in, out->", bin(out_buffer[i]))
#    print("from GS->\t",out1_buffer[i]>>8)#, "\nfrom dummy->\t", out2_buffer[i])

#for i in range(0, 20):
#    print("from Dilate->\t",out2_buffer[i])#, "\nfrom dummy->\t", out2_buffer[i])
#for i in range(100, 120):
#    print("from Gauss->\t",out3_buffer[i])#, "\nfrom dummy->\t", out2_buffer[i])

In [6]:
#out1_buffer.close()
#out2_buffer.close()
#out3_buffer.close()

In [146]:
10*0.3 + 20*0.59 + 30*0.11

18.099999999999998

In [147]:
30*0.3 + 10*0.59 + 20*0.11

17.099999999999998

In [148]:
20*0.3 + 30*0.59 + 10*0.11

24.8

In [100]:
2560*2560

6553600

In [113]:
(6553600)/2**30


0.6103515625

In [117]:
bin(2560>>8)
bin(2560)

'0b101000000000'

In [15]:


#from scipy import ndimage
#from scipy import misc
#f = misc.face()
#misc.imsave('face.png', f) # uses the Image module (PIL)

#import matplotlib.pyplot as plt
#plt.imshow(f)
#plt.show()

#image = open("/home/xilinx/rosetta/image.jpg", "rb")
#while(True):
 #   contents = image.readline()
  #  print(contents)
   # if contents == '':
    #    break
#image.close()

In [13]:
image.close()

In [14]:
bin(97)

'0b1100001'

In [16]:
0.077847*0  + 0.123317*1  + 0.077847*2 + 0.123317*10 + 0.195346*11 + 0.123317*12 + 0.077847*20 + 0.123317*21 + 0.077847*22 

11.000022

In [17]:
  0.077847*1  + 0.123317*2  + 0.123317*3+ 0.123317*11 + 0.195346*12 + 0.123317*13+ 0.077847*21 + 0.123317*22 + 0.077847*23 

12.136434